# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
sdc=school_data_complete

## District Summary

In [3]:
# Calculate the total number of unique schools
sdc.school_name.nunique()

15

In [4]:
# Calculate the total number of students
sdc.student_name.count()

39170

In [5]:
# Calculate the total budget
sdc.groupby('school_name').budget.max().sum()

24649428

In [6]:
sdc.head(2)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635


In [7]:
# Calculate the average (mean) math score
sdc.math_score.mean()

78.98537145774827

In [8]:
# Calculate the average (mean) reading score
sdc.reading_score.mean()

81.87784018381414

In [9]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
sdc[sdc.math_score>69].student_name.count()/sdc.student_name.count()*100

74.9808526933878

In [10]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
sdc[sdc.reading_score>69].student_name.count()/sdc.student_name.count()*100


85.80546336482001

In [11]:
# Use the following to calculate the percentage of students that passed math and reading
sdc[(sdc.math_score>=70)&(sdc.reading_score>=70)].student_name.count()/sdc.student_name.count()*100

65.17232575950983

In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools":[sdc.school_name.nunique()],
                                "Total Students":[sdc.student_name.count()],
                                "Total Budget":[sdc.groupby('school_name').budget.max().sum()],
                                "Average Math Score":[sdc.math_score.mean()],
                                "Average Reading Score":[sdc.reading_score.mean()],
                                "% Passing Math":[sdc[sdc.math_score>69].student_name.count()/sdc.student_name.count()*100],
                                "% Passing Reading":[sdc[sdc.reading_score>69].student_name.count()/sdc.student_name.count()*100],
                                "% Overall Passing":[sdc[(sdc.math_score>=70)&(sdc.reading_score>=70)].student_name.count()/sdc.student_name.count()*100]})
                                                        

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [13]:
# Use the code provided to select the school type

school_types = school_data.set_index(["school_name"])["type"]

In [37]:
# Calculate the total school budget and per capita spending
print(f'Total School Budget: ${sdc.groupby("school_name").budget.max().sum():,}')
print(f'Per Capita: ${sdc.groupby("school_name").budget.max().sum()/sdc.student_name.count():,.2f}')

Total School Budget: $24,649,428
Per Capita: $629.29


In [68]:
# Calculate the average test scores
school_type = sdc.groupby("school_name").type.unique()
total_per_school_budget = sdc.groupby("school_name").budget.mean()
per_school_average_math = sdc.groupby("school_name").math_score.sum()
per_school_average_reading = sdc.groupby("school_name").reading_score.sum()
per_school_total_student= sdc.groupby("school_name").student_name.count()
per_student_budget = total_per_school_budget/per_school_total_student
per_school_math = sdc[sdc.math_score>=70].groupby("school_name").student_name.count()
per_school_reading = sdc[sdc.reading_score>=70].groupby("school_name").student_name.count()
per_school_reading_math =sdc[(sdc.reading_score>=70)&(sdc.math_score>=70)].groupby("school_name").student_name.count()
percent_passing_math = per_school_math/per_school_total_student*100
percent_passing_reading =per_school_reading/per_school_total_student*100
percent_passing_overall = per_school_reading_math/per_school_total_student*100

grp = sdc.groupby('school_name')
mPc = sdc[sdc.math_score>69].groupby('school_name').student_name.count()
rPc = sdc[sdc.reading_score>69].groupby('school_name').student_name.count()
aPc = sdc[(sdc.reading_score>69)&(sdc.math_score>69)].groupby('school_name').student_name.count()


school_summary = pd.DataFrame({ "School type":school_type,
                                "Total Students":(per_school_total_student).map('{:,}'.format),
                                 "Total School Budget":(total_per_school_budget).map('${:,.2f}'.format),
                                "Per Student Budget":(per_student_budget).map('{:.0f}'.format),
                               "Average Math Score": (grp.math_score.mean()).map('{:.3f}'.format),
                               "Average Reading Score": (grp.reading_score.mean()).map('{:.3f}'.format),
                               "% Passing Math": (mPc/grp.student_name.count()*100).map('{:.3f}%'.format),
                               "% Passing Reading": (rPc/grp.student_name.count()*100).map('{:.3f}%'.format),
                               "% Overall Math": (aPc/grp.student_name.count()*100).map('{:.3f}%'.format)
                             })
school_summary


,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Math
school_name,,,,,,,,,
Bailey High School,[District],"4,976","$3,124,928.00",628,77.048,81.034,66.680%,81.933%,54.642%
Cabrera High School,[Charter],"1,858","$1,081,356.00",582,83.062,83.976,94.133%,97.040%,91.335%
Figueroa High School,[District],"2,949","$1,884,411.00",639,76.712,81.158,65.988%,80.739%,53.204%
Ford High School,[District],"2,739","$1,763,916.00",644,77.103,80.746,68.310%,79.299%,54.290%
Griffin High School,[Charter],"1,468","$917,500.00",625,83.351,83.817,93.392%,97.139%,90.599%
Hernandez High School,[District],"4,635","$3,022,020.00",652,77.290,80.934,66.753%,80.863%,53.528%
Holden High School,[Charter],427,"$248,087.00",581,83.803,83.815,92.506%,96.253%,89.227%
Huang High School,[District],"2,917","$1,910,635.00",655,76.629,81.183,65.684%,81.316%,53.514%
Johnson High School,[District],"4,761","$3,094,650.00",650,77.072,80.966,66.058%,81.222%,53.539%


In [39]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.


# Formatting
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
school_summary

,School type,Total Students,Total School Budget,Per Student Budget
school_name,,,,
Bailey High School,[District],4976,"$3,124,928.00",$628.00
Cabrera High School,[Charter],1858,"$1,081,356.00",$582.00
Figueroa High School,[District],2949,"$1,884,411.00",$639.00
Ford High School,[District],2739,"$1,763,916.00",$644.00
Griffin High School,[Charter],1468,"$917,500.00",$625.00
Hernandez High School,[District],4635,"$3,022,020.00",$652.00
Holden High School,[Charter],427,"$248,087.00",$581.00
Huang High School,[District],2917,"$1,910,635.00",$655.00
Johnson High School,[District],4761,"$3,094,650.00",$650.00


## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
 

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.


## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = 
tenth_graders_scores = 
eleventh_graders_scores = 
twelfth_graders_scores = 

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`


# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = 
tenth_graders_scores = 
eleventh_graders_scores = 
twelfth_graders_scores = 

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`


# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`


# Display results
type_summary